In [2]:
# importing libraries
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.optimizers import SGD
from keras.utils import to_categorical
import random
import pickle
import json
import nltk
from nltk.stem import WordNetLemmatizer

ModuleNotFoundError: No module named 'tensorflow'

: 

In [ ]:
# Tokenizing and lemmatizing
lemmatizer = WordNetLemmatizer()
words, classes, documents = [], [], []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

In [ ]:
documents

In [31]:

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in doc[0]]
    
    # create our bag of words array with 1, if word match found in current pattern
    bag = [1 if w in pattern_words else 0 for w in words]
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [ ]:
# Shuffle the training data
random.shuffle(training)

# Ensure each part of the training data is converted to NumPy arrays correctly
train_x = np.array([np.array(x, dtype=np.float32) for x, y in training])
train_y = np.array([np.array(y, dtype=np.float32) for x, y in training])

print("Training data created")

In [ ]:
# Create model - 3 layers
model = Sequential()

# Input layer using Input() object
model.add(Input(shape=(len(train_x[0]),)))

# First hidden layer with 128 neurons
model.add(keras.layers.Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Second hidden layer with 64 neurons
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

# Output layer with neurons equal to number of intents (i.e., number of classes)
model.add(Dense(len(classes), activation='softmax'))

# Compile model with SGD optimizer
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

print("Model created successfully")

In [ ]:
print(type(train_x))
print(train_x.shape)
print(train_x[0])


In [ ]:
train_x = np.array(train_x, dtype=np.float32)
train_x

In [ ]:
print(type(train_y))
print(train_y.shape)
print(train_y[0])

In [ ]:
# Fit the model
hist = model.fit(train_x, train_y, epochs=500, batch_size=5, verbose=1)

# Save the model
model.save('model.h5')
print("Model created and saved")
